In [1]:
import numpy as np
from mne.decoding import CSP

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
import sys

In [5]:
sys.path.append('../models/')

In [11]:
from Extract_data import Extract_data_from_subject
from preprocessing import Select_time_window, csp_transform, bandpass_filter
from TOL_dataset_utils import Transform_for_classificator
from constants import *
from models.NN import Net
from loops import *

In [12]:
scaler = MinMaxScaler()
loss_fn = nn.CrossEntropyLoss()

In [13]:
def save_results_for_participant(module, model_name, results_save, save_dir, Condition, subject, accuracy_list):
     if results_save:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        file_name = save_dir + "_" + module + "_" + model_name + "_AVG_CV_" + Condition  + "_Subject_" + str(subject)+ ".npy"

        np.save(file_name, accuracy_list)

## Klasifikacija

In [18]:
for subject in SUBJECT_NUMBERS:

    X, y = Extract_data_from_subject(ROOT, subject, DATA_TYPE)
    # vreme kada postoji nadrazaj
    X = Select_time_window(X, START_T, END_T, SAMPLE_FREQ)
    # izvlacenje samo covert trial-a za sva 4 pravca
    X, y = Transform_for_classificator(X, y, CONDITIONS, CLASSES)

    X = bandpass_filter(bands, SAMPLE_FREQ, X, DEVICE)

    outer_cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)


    model = Net()
    model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR_STEP_SIZE)


    # store avg accuracy for each fold
    cv_metrics_log = [0 for i in range(N_SPLITS)]

    # CV
    for i, (train_indices, test_indices) in enumerate(outer_cv.split(X, y)):

        # Pretprocesiranje
        y_train, y_test = y[train_indices], y[test_indices]
        X_train, X_test = X[train_indices], X[test_indices]
        X_train, X_test = csp_transform(X_train, y_train, X_test, csp) # reshapes so scaler is applicable
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Tenzori
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.int64)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        y_test = torch.tensor(y_test, dtype=torch.int64)

        # Create datasets for cross validation
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = False)
        test_dataloader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

        for _ in range(EPOCHS):
          train_loop(train_dataloader, model, loss_fn, optimizer, DEVICE)
          cv_metrics_log[i] += test_loop(test_dataloader, model, loss_fn, DEVICE)
        
        # avg accuracy for current fold
        cv_metrics_log[i] /= EPOCHS



    # end of CV loop
    best_accuracy = max(cv_metrics_log)
    avg_model_accuracy = sum(cv_metrics_log)/N_SPLITS
    print(f'Subject: {subject} NN: Avg accuracy: {avg_model_accuracy} Best accuracy: {best_accuracy}')
    save_results_for_participant("TORCH", "NN", SAVE, SAVE_DIR, CONDITIONS[0], subject, avg_model_accuracy)

/Users/jovan/Fax/Treca_godina/RI/Projekat/Kod/preprocessing.py:27: RuntimeWarning: filter_length (1677) is longer than the signal (508), distortion is likely. Reduce filter length or filter a longer signal.
  X_filtered = mne.filter.filter_data(X, sample_freq, freq_low, freq_high, n_jobs = n_jobs, verbose = False)
/Users/jovan/miniconda3/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Computing rank from data with rank=None
    Using tolerance 0.0018 (2.2e-16 eps * 768 dim * 1.1e+10  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 768 dim * 9.7e+09  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0018 (2.2e-16 eps * 768 dim * 1.1e+10  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0021 (2.2e-16 eps * 768 dim * 1.2e+10  max s

/Users/jovan/Fax/Treca_godina/RI/Projekat/Kod/preprocessing.py:27: RuntimeWarning: filter_length (1677) is longer than the signal (508), distortion is likely. Reduce filter length or filter a longer signal.
  X_filtered = mne.filter.filter_data(X, sample_freq, freq_low, freq_high, n_jobs = n_jobs, verbose = False)


KeyboardInterrupt: 